# Bode Plot of LTI Systems

Signals- & Systems, University of Rostock, [Institute of Communications Engineering](https://www.int.uni-rostock.de/), Prof. [Sascha Spors](https://orcid.org/0000-0001-7225-9992), [Frank Schultz](https://orcid.org/0000-0002-3010-0294), [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

### References:

* Norbert Fliege (1991): "*Systemtheorie*", Teubner, Stuttgart (GER), cf. chapter 4.3.5

* Alan V. Oppenheim, Alan S. Willsky with S. Hamid Nawab (1997): "*Signals & Systems*", Prentice Hall, Upper Saddle River NJ (USA), 2nd ed., cf. chapter 6

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2001): "*Signals and Systems*", Wiley, Chichester (UK), cf. chapter 10

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2005/2007): "*Einführung in die Systemtheorie*", Teubner, Wiesbaden (GER), 3rd/4th ed., cf. chapter 10

Let's define some helping routines for plotting in the following.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal

def lti_bode_plot(sys, txt):
    w, mag, phase = sys.bode(np.arange(1e-2,1e2,1e-2))
    fig = plt.figure(figsize=(9, 3))
    plt.title(txt)
    plt.semilogx(w,mag,'C0', linewidth=3)
    plt.grid(True)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('Magnitude: abs(H) / dB')
    plt.xlim(1e-2,1e+2)
    plt.ylim(-40,40)
    plt.yticks(np.arange(-40,50,10))    
    plt.figure(figsize=(9, 3))
    plt.semilogx(w,phase,'C3', linewidth=3)
    plt.grid(True)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('Phase: arg(H) / deg')
    plt.xlim(1e-2,1e+2)
    plt.ylim(-225,225)
    plt.yticks(np.arange(-180,225,45))
    
#plot LTI system characteristics for Laplace domain
def plot_LTIs(B, A):

    w, H = signal.freqs(B, A, 2**10)
    th, h = signal.impulse([B, A])
    the, he = signal.step([B, A])
    zeros = np.roots(B)
    poles = np.roots(A)

    plt.figure(figsize=(12,12))

    #pole / zero plot
    plt.subplot(321)
    plt.plot(np.real(poles), np.imag(poles),'x', color='C0')
    plt.plot(np.real(zeros), np.imag(zeros),'o', color='C0', mfc='none')
    plt.axis("equal")
    plt.xlabel('Re(s)')
    plt.ylabel('Im(s)')
    plt.grid(True)
    plt.title("Pole/Zero Map")
    
    #Nyquist plot
    plt.subplot(322)
    plt.plot(H.real, H.imag, "C0", label="$\omega>0$")
    plt.plot(H.real, -H.imag, "C3", label="$\omega<0$")
    plt.plot(H.real[0], H.imag[0], marker='$w=0$', markersize=25, color="C0")
    plt.axis("equal")
    plt.legend()
    plt.xlabel('Re(H)')
    plt.ylabel('Im(H)')
    plt.grid(True)
    plt.title("Nyquist Plot")
    
    #magnitude response
    plt.subplot(323)
    plt.semilogx(w, 20*np.log10(np.abs(H)))
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('A / dB')
    plt.grid(True)
    plt.title("Level")
    plt.xlim((w[0], w[-1]))
    plt.ylim((-60,6))
    plt.yticks(np.arange(-60,+12,6))
    
    #phase response
    plt.subplot(324)
    plt.semilogx(w, np.angle(H)*180/np.pi)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('phi / deg')
    plt.grid(True)
    plt.title("Phase")
    plt.xlim((w[0], w[-1]))
    plt.ylim((-180,+180))
    plt.yticks(np.arange(-180,+210,30))
    
    
    #impulse response
    plt.subplot(325)
    plt.plot(th, h)
    plt.xlabel('t / s')
    plt.ylabel('h(t)')
    plt.grid(True)
    plt.title("Impulse Response")
    
    #step response
    plt.subplot(326)
    plt.plot(the, he)
    plt.xlabel('t / s')
    plt.ylabel('h$_\epsilon$(t)')
    plt.grid(True)
    plt.title("Step Response")    

#plot LTI system characteristics for z-domain
def plot_LTIz(b, a):
    TBD = true
        

# Example: Unity Gain

In [ ]:
sz = 0
sp = 0
k = +1
txt = 'Unity Gain'
sys = signal.lti(sz, sp, k)
lti_bode_plot(sys, txt)

# Example: Gain and Polarity

In [ ]:
sz = 0
sp = 0
k = -10
txt = '20 dB Gain with Inverted Polarity'
sys = signal.lti(sz, sp, k)
lti_bode_plot(sys, txt)

# Example: Poles / Zeros in Origin

In [ ]:
sz = 0, 0,  #note: more zeros than poles is not a causal system!
sp = 0,
k = 1
sys = signal.lti(sz, sp, k)
txt = str(len(sz)) + ' Zeros / ' + str(len(sp)) + ' Poles in Origin'
txt1 = (': ' + str((len(sz)-len(sp))*20) + ' dB / decade') 
lti_bode_plot(sys, txt+txt1)

In [ ]:
sz = 0,
sp = 0, 0, 0
k = 1
sys = signal.lti(sz, sp, k)
txt = str(len(sz)) + ' Zeros / ' + str(len(sp)) + ' Poles in Origin'
txt1 = (': ' + str((len(sz)-len(sp))*20) + ' dB / decade') 
lti_bode_plot(sys, txt+txt1)

# Example: Single Real Pole

In [ ]:
sz = 0
sp = 0, -1
k = 1
sys = signal.lti(sz, sp, k)
txt = 'Single Real Pole, decreasing slope, -20 dB / decade'
lti_bode_plot(sys, txt)

# Example: Single Real Zero

In [ ]:
sz = 0, -1
sp = 0
k = 1
sys = signal.lti(sz, sp, k)
txt = 'Single Real Zero, increasing slope, + 20 dB / decade'
lti_bode_plot(sys, txt)

# Example: Complex Conjugate Zero Pair

In [ ]:
sz = 0, -3/4-1j, -3/4+1j
sp = 0
k = 16/25
sys = signal.lti(sz, sp, k)
txt = 'Conjugate Complex Zero'
lti_bode_plot(sys, txt)

# Example: Complex Conjugate Pole Pair

In [ ]:
# this is the ODE RLC-example from 'solving_2nd_order_ode.pdf':
# 16/25 y''(t) + 24/25 y'(t) + y(t) = DiracDelta(t), y'(t=0)=0, y(t=0)=0

# B = (0., 0., 1.)
# A = (16/25, 24/25, 1)
# sys = signal.lti(B, A)
# txt = 'Conjugate Complex Pole, -3/4$\pm$1j'
# lti_bode_plot(sys, txt)
# equal to:

sz = 0
sp = 0, -3/4-1j, -3/4+1j
k = 25/16
sys = signal.lti(sz, sp, k)
txt = 'Conjugate Complex Pole, -3/4$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = 0
sp = 0, -1/2-1j, -1/2+1j
k = 5/4
sys = signal.lti(sz, sp, k)
txt = 'Conjugate Complex Pole, -1/2$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = 0
sp = 0, -1/4-1j, -1/4+1j
k = 17/16
sys = signal.lti(sz, sp, k)
txt = 'Conjugate Complex Pole, -1/4$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = 0
sp = 0, -1/8-1j, -1/8+1j
k = 65/64
sys = signal.lti(sz, sp, k)
txt = 'Conjugate Complex Pole, -1/8$\pm$1j'
lti_bode_plot(sys, txt)

# Example: Simple Bandpass from Real Poles
* zero in origin $s_0=0$
* pole at $s_{\infty,1}=-0.1$
* pole at $s_{\infty,2}=-10$
* $H_0$ = 10

\begin{align}
H(s) = H_0\frac{s-s_{0,1}}{(s-s_{\infty,1})(s-s_{\infty,2})} = 10\frac{(s-0)}{(s+0.1)(s+10)}
=\frac{100 s}{10 s^2 + 101 s + 10}
\end{align}

In [ ]:
sz = 0, 0
sp = 0, -0.1, -10
H0 = 10
sys = signal.lti(sz, sp, H0)
txt = 'Bandpass'
lti_bode_plot(sys, txt)
#equal to
#B = (0,100,0)
#A = (10, 101, 10)
#sys = signal.lti(B, A)
#txt = 'Bandpass'

#lti_bode_plot(sys, txt)

### Lowpass

The 2nd order lopwass
\begin{align}
H_\mathrm{Low}(s) = \frac{1}{\frac{16}{25}s^2+\frac{24}{25}s +1} = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}
\end{align}
is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
if False:
    # lowpass 2nd order, w0 = 1, D = 1/sqrt(2) -> Q = 1/sqrt(2)
    # this is equivalent to a 2nd order Butterworth
    # B,A = signal.butter(2, 1, btype='low', analog=True)
    w0 = 1
    D = np.sqrt(2)/2
else:    
    # this is the ODE RLC-example from 'solving_2nd_order_ode.pdf':
    # 16/25 y''(t) + 24/25 y'(t) + y(t) = DiracDelta(t), y'(t=0)=0, y(t=0)=0
    # D = 3/5, w0 = 5/4, sigma0 = -3/4, wD = 1
    A = (16/25, 24/25, 1)
    print("A = ", A)
    w0 = 5/4
    D = 3/5

Q = 1/(2*D)    
A = (1/w0**2, 2*D/w0, 1)
print("A = ", A)
A = (1/w0**2, 1/(Q*w0), 1)
print("A = ", A)
B = (0, 0, 1)
print("D = ", D, "Q = ", Q)
plot_LTIs(B, A)

### Lowpass to Highpass Transform
The Laplace transfer function of a lowpass is transformed to a highpass by exchanging
\begin{align}
s \rightarrow \frac{1}{s}
\end{align}

The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields the
following 2nd order highpass filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
#highpass 2nd order, from lowpass 2nd order with s -> 1/s
A = (1, 24/25, 16/25)
B = (1, 0, 0)
plot_LTIs(B, A)

### Lowpass to Bandpass Transform

The Laplace transfer function of a lowpass is transformed to a bandpass by exchanging 
\begin{align}
s \rightarrow s + \frac{1}{s}
\end{align}
The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields to the
following 4th order bandpass filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
#bandpass 4th order, from lowpass 2nd order with s -> s + 1/s
A = (16, 24, 57, 24, 16)
B = (0,0, 25, 0, 0)
plot_LTIs(B, A)

### Lowpass to Bandstop Transform

The Laplace transfer function of a lowpass is transformed to a bandstop filter by exchanging
\begin{align}
s \rightarrow \frac{1}{s + \frac{1}{s}}
\end{align}

The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields to the
following 4th order bandstop filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
#bandstop 4th order, from lowpass 2nd order with s -> 1 / (s + 1/s)
A = (25, 24, 66, 24, 25)
B = (25, 0, 50, 0, 25)
plot_LTIs(B, A)